In [1]:
from __future__ import print_function

import os
import sys
import timeit
import numpy

import theano
import theano.tensor as T
from theano.tensor.signal import pool
from theano.tensor.nnet import conv2d


class ConvPoolLayer(object):
    """Pool Layer of a convolutional network """

    def __init__(self, rng, input, filter_shape, image_shape, stride=(2, 2)):
        """

        """

        assert image_shape[1] == filter_shape[1]
        self.input = input
        # there are "num input feature maps * filter height * filter width"
        # inputs to each hidden unit
        fan_in = numpy.prod(filter_shape[1:])
        # each unit in the lower layer receives a gradient from:
        # "num output feature maps * filter height * filter width" /
        #   pooling size
        fan_out = (filter_shape[0] * numpy.prod(filter_shape[2:]) //
                   numpy.prod(stride))
        # initialize weights with random weights
        W_bound = numpy.sqrt(6. / (fan_in + fan_out))
        self.W = theano.shared(
            numpy.asarray(
                rng.uniform(low=-W_bound, high=W_bound, size=filter_shape),
                dtype=theano.config.floatX
            ),
            borrow=True
        )

        # the bias is a 1D tensor -- one bias per output feature map
        b_values = numpy.zeros((filter_shape[0],), dtype=theano.config.floatX)
        self.b = theano.shared(value=b_values, borrow=True)

        # convolve input feature maps with filters
        conv_out = conv2d(
            input=input,
            filters=self.W,
            filter_shape=filter_shape,
            input_shape=image_shape,
            subsample=stride
        )

        # pool each feature map individually, using maxpooling
        '''pooled_out = pool.pool_2d(
            input=conv_out,
            ds=poolsize,
            ignore_border=True
        )'''

        # add the bias term. Since the bias is a vector (1D array), we first
        # reshape it to a tensor of shape (1, n_filters, 1, 1). Each bias will
        # thus be broadcasted across mini-batches and feature map
        # width & height
        self.output = T.tanh(conv_out + self.b.dimshuffle('x', 0, 'x', 'x'))

        # store parameters of this layer
        self.params = [self.W, self.b]

        # keep track of model input
        self.input = input


In [15]:
class SDLayer1(object):
    """Pool Layer of a convolutional network """

    def __init__(self, rng, input, filter_shape, image_shape, stride=(2, 2)):
        """

        """

        assert image_shape[1] == filter_shape[1]
        self.input = input
        # there are "num input feature maps * filter height * filter width"
        # inputs to each hidden unit
        fan_in = numpy.prod(filter_shape[1:])
        # each unit in the lower layer receives a gradient from:
        # "num output feature maps * filter height * filter width" /
        #   pooling size
        fan_out = (filter_shape[0] * numpy.prod(filter_shape[2:]) //
                   numpy.prod(stride))
        # initialize weights with random weights
        W_bound = numpy.sqrt(6. / (fan_in + fan_out))
        self.W = theano.shared(
            numpy.asarray(
                rng.uniform(low=-W_bound, high=W_bound, size=filter_shape),
                dtype=theano.config.floatX
            ),
            borrow=True
        )

        # the bias is a 1D tensor -- one bias per output feature map
        b_values = numpy.zeros((filter_shape[0],), dtype=theano.config.floatX)
        self.b = theano.shared(value=b_values, borrow=True)

        # convolve input feature maps with filters
        conv_out = conv2d(
            input=input,
            filters=self.W,
            filter_shape=filter_shape,
            input_shape=image_shape,
            subsample=stride
            )
        # add the bias term. Since the bias is a vector (1D array), we first
        # reshape it to a tensor of shape (1, n_filters, 1, 1). Each bias will
        # thus be broadcasted across mini-batches and feature map
        # width & height
        self.output = T.nnet.sigmoid(conv_out + self.b.dimshuffle('x', 0, 'x', 'x'))

        # store parameters of this layer
        self.params = [self.W, self.b]

        # keep track of model input
        self.input = input

In [16]:
class SDLayer2(object):
    """SPACEDISPLACEMENTLOCALIZATION"""

    def __init__(self, input, image_shape, numberN):

        self.input = input
        delta_x = 598/2
        delta_y = 838/33
        tilda_x = 598/2
        tilda_y = 838/33
        
        offset_y=numpy.asarray([range(image_shape[3])]*image_shape[2])*delta_y
        offset_x=(numpy.asarray([range(image_shape[2])]*image_shape[3])*delta_x).T
        zeroArray=numpy.zeros(image_shape[2:])
        offsetValue = numpy.asarray([zeroArray,offset_x,offset_y]*numberN)
        offsetAdd =  numpy.asarray([offsetValue]*image_shape[0],dtype=theano.config.floatX)
        t_offset = theano.shared(offsetAdd)
        onsArray=numpy.ones(image_shape[2:])
        scale_x=onsArray*tilda_x
        scale_y=onsArray*tilda_y
        scaleValue = numpy.asarray([onsArray,scale_x,scale_y]*numberN)
        scales =  numpy.asarray([scaleValue]*image_shape[0],dtype=theano.config.floatX)

        t_scale = theano.shared(scales)
        
        self.output = self.input*t_scale+t_offset

        # store parameters of this layer
        self.params = []

        # keep track of model input
        self.input = input


In [17]:
     
# main
# create a np array of size 1×(598×838)
inputs=numpy.random.rand(10,598,838)
layer0_input = inputs.reshape((10, 1, 598,838))
batch_size =10
rng = numpy.random.RandomState(23455)

x = T.dtensor4('x') 

layer1 = ConvPoolLayer(rng,
        input=x,
        image_shape=(batch_size, 1, 598, 838),
        filter_shape=(12, 1, 4, 4),
        stride=(3, 3))
layer2 = ConvPoolLayer(rng,
        input=layer1.output,
        image_shape=(batch_size,12, 199, 279),
        filter_shape=(16, 12, 4, 3),
        stride=(3, 2))
layer3 = ConvPoolLayer(rng,
        input=layer2.output,
        image_shape=(batch_size, 16, 66, 139),
        filter_shape=(24, 16, 6, 3),
        stride=(4, 2))
layer4 = ConvPoolLayer(rng,
        input=layer3.output,
        image_shape=(batch_size, 24, 16, 69),
        filter_shape=(30, 24, 4, 3),
        stride=(3, 2))
layer5 = ConvPoolLayer(rng,
        input=layer4.output,
        image_shape=(batch_size, 30, 5, 34),
        filter_shape=(36, 30, 3, 2),
        stride=(2, 1))
numberN=20
'''the layer 6 and 7 together called SDL layer'''
layer6 = SDLayer1(rng,
        input=layer5.output,
        image_shape=(batch_size, 36, 2, 33),
        filter_shape=(3*numberN, 36, 1, 1),
        stride=(1, 1))
layer7 = SDLayer2(input = layer6.output,
                image_shape = (batch_size,3*numberN,2,33),
                numberN =numberN)
vis_layer1 = theano.function([x], [layer7.output])
kk = vis_layer1(layer0_input)
print(kk[0][0][0])

(1, 1)
[[ 0.23701806  0.22777578  0.23879169  0.30651516  0.27771995  0.2511339
   0.24834874  0.23634486  0.29555638  0.26715209  0.26527429  0.24695437
   0.23442412  0.33852737  0.24371851  0.25787364  0.287042    0.30543206
   0.25080042  0.30617941  0.20904458  0.22915916  0.27057243  0.2452865
   0.26082988  0.28671771  0.24803737  0.23961328  0.20697444  0.2459569
   0.30382111  0.26973355  0.22349732]
 [ 0.21699375  0.2836362   0.23571075  0.3043387   0.26768557  0.23858907
   0.21525826  0.24242648  0.20764267  0.26583276  0.3207926   0.24311386
   0.23125793  0.32233652  0.36486167  0.2120372   0.25490551  0.31248127
   0.25401542  0.22474844  0.31545602  0.23660251  0.27503323  0.32002071
   0.25571893  0.28410637  0.26225125  0.31103953  0.31051156  0.26596543
   0.22536954  0.26400193  0.22348902]]
